# Make Mercury from an Enstatite chondrite composition

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Readin EH composition file

In [36]:
# Read in 
dfEc = pd.read_csv('eh_comps.csv',header=0,index_col=0,na_values='nan')

## Mass balance

In [ ]:
# set masses for each layer

masses = {'core':0.670, 'core_err':0.03}
# constrain silicate mercury by difference
masses['silicate'] = 1 - masses['core']

# vector of atomic masses
# meeds to match the elemental array passed in for planetary mass balance calculations
atmasses = {'O': 15.999,'Si': 28.084, 'Mg': 24.305, 'Al': 26.982, 'Ca': 40.078, 'Fe': 55.845,
            'Ni': 58.693, 'S': 32.06, 'Ti': 47.867, 'Mn': 54.938, 'Cr': 51.996, 'Co': 58.933,
            'Na': 22.990, 'K': 39.098}

In [ ]:
class Planet:
    def __init__(self, reservoirs, components, mix, mod_components=None):
        # reservoirs is a dictionary of reservoirs and their mass fractions
        self.reservoirs = reservoirs
        # components is a dictionary of component compositions that will be mixed to make the planet
        # compositions in weight fraction
        self.components = components
        # this is the mixing matrix, that should relate each reservoir to a mixture of components
        # if we wanted to make pseudo compoments (e.g., taking Si out of silicate and into the core)
        # then we should add a component modification matrix
        self.mix = mix

        # this is the object that we would use to modify the component compositions
        #  it is structured as a dictionary of dictionaries
        #  the outer dictionary is the component we are modifying
        #  the inner dictionaries indicate how the other components take the moles from the modified component
        #  all loss/gain is on a molar basis
        self.mod_components = mod_components
        




In [ ]:
# mixing matrix mapping reservoirs to components
mix = {'core'  : {'metal':1, 'silicate':0, 'sulfide':0},
       'mantle': {'metal':0, 
                  'silicate':dfEc.loc['silicate'].prop/(dfEc.loc['silicate'].prop + dfEc.loc['sulfide'].prop), 
                  'sulfide': dfEc.loc['sulfide'].prop/(dfEc.loc['silicate'].prop + dfEc.loc['sulfide'].prop)}}

# component modification matrix
#  the inner dictionaries of arrays describe the fraction of moles lost from each component of each element
#  the inner dictionaries of arrays, for the component not being modified, should sum to 1.
#  e.g., if we modify silicate, then the metal + sulfide arrays should sum to 1 for each element
mod_componentds = {'silicate': {'silicate': np.zeros(len(atmasses)), 'metal': np.zeros(len(atmasses)), 'sulfide': np.zeros(len(atmasses))},
                   'metal'   : {'silicate': np.zeros(len(atmasses)), 'metal': np.zeros(len(atmasses)), 'sulfide': np.zeros(len(atmasses))},
                   'sulfide' : {'silicate': np.zeros(len(atmasses)), 'metal': np.zeros(len(atmasses)), 'sulfide': np.zeros(len(atmasses))}}

merc = Planet(masses, 
              {'silicate': dfEc.loc['silicate'], 'metal': dfEc.loc['metal'], 'sulfide': dfEc.loc['sulfide']},
              mix)

In [40]:
merc.mix

{'core': {'metal': 1, 'silicate': 0, 'sulfide': 0},
 'mantle': {'metal': 0,
  'silicate': np.float64(0.8548185231539425),
  'sulfide': np.float64(0.1451814768460576)}}